## Importing packages and csvs

In [2]:
# importing packages
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine
from ET_mysql_key import ET_key
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# setting up csv variables
WSJ_salaries_college_file = "../Data_Files/college_salaries/salaries-by-college-type.csv" # has median salary data 
CC_institution_details_file = "../Data_Files/college_completion/cc_institution_details.csv" # has all data incl. school name
CC_institution_grads_file = "../Data_Files/college_completion/cc_institution_grads.csv" # has gender data
# WSJ_salaries_region_file = "../Data_Files/college_salaries/salaries-by-region.csv"
# CC_state_sector_details_file = "../Data_Files/college_completion/cc_state_sector_details.csv"
# CC_state_sector_grads_file = "../Data_Files/college_completion/cc_state_sector_grads.csv"

# testing 
# PEPS_closed_schools_file = "/Data_Files/closedschoolsearch.csv"

## Extraction

In [4]:
# extracting csvs into dataframes
WSJ_college_raw_df = pd.read_csv(WSJ_salaries_college_file)
# WSJ_college_raw_df.head()

In [5]:
CC_institution_details_raw_df = pd.read_csv(CC_institution_details_file)
# CC_institution_details_raw_df.head(2)
# CC_institution_details_raw_df.shape

In [6]:
CC_institution_grads_raw_df = pd.read_csv(CC_institution_grads_file)
# CC_institution_grads_raw_df.head()

## Cleaning

In [7]:
# remove columns as needed and set as the real df
WSJ_college_cols = ["School Name", "School Type", "Starting Median Salary", "Mid-Career Median Salary"]
WSJ_college_df = WSJ_college_raw_df[WSJ_college_cols].copy()
WSJ_college_df = WSJ_college_df.rename(columns={"School Name": "school_name", "School Type": "school_type", 
                                                "Starting Median Salary": "median_salary_start", 
                                                "Mid-Career Median Salary": "median_salary_mid"})
WSJ_college_Df = WSJ_college_df.dropna(how="any")
WSJ_college_Df.head()

,school_name,school_type,median_salary_start,median_salary_mid
0,Amherst College,Liberal Arts,"$54,500.00","$107,000.00"
1,Appalachian State University,State,"$40,400.00","$69,100.00"
2,Arizona State University,Party,"$47,400.00","$84,100.00"
3,Arizona State University,State,"$47,400.00","$84,100.00"
4,Arkansas State University,State,"$38,700.00","$63,300.00"


In [8]:
# remove columns as needed and set as the real df
CC_institution_details_cols = ["unitid", "chronname", "state", "level","control","basic",
                               "student_count","aid_value","grad_100_value","grad_100_percentile",
                              "grad_150_value","grad_150_percentile"]
CC_institution_details_df = CC_institution_details_raw_df[CC_institution_details_cols].drop_duplicates().copy()
CC_institution_details_df = CC_institution_details_df.rename(columns={"unitid": "school_id", 
                                                                      "chronname":"school_name"})
CC_institution_details_Df = CC_institution_details_df.dropna(how="any")
CC_institution_details_Df.head()

,school_id,school_name,state,level,control,basic,student_count,aid_value,grad_100_value,grad_100_percentile,grad_150_value,grad_150_percentile
0,222178,Abilene Christian University,Texas,4-year,Private not-for-profit,Masters Colleges and Universities--larger prog...,3806,9405.0,37.0,44.0,56.9,53.0
2,172866,Academy College,Minnesota,4-year,Private for-profit,Baccalaureate and Associates Colleges,236,4564.0,100.0,97.0,100.0,96.0
3,108232,Academy of Art University,California,4-year,Private for-profit,Schools of art- music- and design,12181,5342.0,5.6,36.0,29.2,58.0
4,441201,Academy of Court Reporting at Cincinnati,Ohio,2-year,Private for-profit,Associates--Private For-profit,149,4898.0,0.0,0.0,0.0,0.0
5,200633,Academy of Court Reporting at Cleveland,Ohio,2-year,Private for-profit,Associates--Private For-profit,399,5544.0,35.7,41.0,35.7,15.0


In [9]:
# removing year column and dropping duplicates
CC_institution_grads_cols = ["unitid", "gender", "race", "cohort"]
CC_institution_grads_df = CC_institution_grads_raw_df[CC_institution_grads_cols].drop_duplicates().copy()
CC_institution_grads_df = CC_institution_grads_df.rename(columns={"unitid":"school_id"})
CC_institution_grads_Df = CC_institution_grads_df.dropna(how="any")
CC_institution_grads_Df.head()

,school_id,gender,race,cohort
0,101462,B,X,2y all
1,101471,B,X,2y all
2,101499,B,X,2y all
3,101505,B,X,2y all
4,101514,B,X,2y all


## Loading

In [ ]:
# creating database connection 
# ET_key for ET's MySQL access only
connection_string = "root:{ET_key}@localhost/Colleges"
engine = create_engine(f'mysql://{connection_string}')

In [ ]:
# confirming tables 
engine.table_names()

In [ ]:
# loading dataframes into database
WSJ_college_Df.to_sql(name='Salaries', con=engine, if_exists='replace', index=True)

CC_institution_details_Df.to_sql(name='Grad_Rates', con=engine, if_exists='replace', index=True)

CC_institution_grads_Df.to_sql(name='Demographics', con=engine, if_exists='replace', index=True)

In [ ]:
# confirming that tables loaded successfully
pd.read_sql("select * from Demographics limit 10",con = engine)